In [10]:
using DataFrames, Distributions, DataFramesMeta

In [12]:
function gen_data(; n = 500000, mu = [0, 2], sigma = 1, pZ1 = 0.8)
    data = DataFrame(
        Obs_ID = 1:n,
        Z = (rand(Uniform(0,1), n) .> pZ1) .+ 1
        )

    @transform!(data, :Y = rand(Normal(0, sigma), n) + mu[:Z])    
    @select!(data, :Obs_ID, :Y)
    @transform!(data, :p_Y_given_Z_1 = pdf.(Normal(mu[1], sigma), :Y))
    @transform!(data, :p_Y_given_Z_2 = pdf.(Normal(mu[2], sigma), :Y))

    return data
  end

gen_data (generic function with 1 method)

In [13]:
data = gen_data(n = 500000, pZ1 = 0.8);
first(data, 6)

,Obs_ID,Y,p_Y_given_Z_1,p_Y_given_Z_2
,Int64,Float64,Float64,Float64
1,1,0.000456794,0.398942,0.0540403
2,2,1.56857,0.116584,0.36349
3,3,0.790284,0.291938,0.191926
4,4,-1.76597,0.0838889,0.000332059
5,5,-0.394954,0.36901,0.0226671
6,6,-1.09829,0.218263,0.00328423


In [4]:
function fit_model!(
    data; 
    pi_hat_0 = 0.5, 
    tolerance = 0.0001,
    max_iterations = 1000,
    progress = DataFrame(
        iter = 1:(max_iterations+1), 
        pi_hat = Vector{Float64}(undef, max_iterations+1), 
        ll = Vector{Float64}(undef, max_iterations+1), 
        ll_diff = Vector{Float64}(undef, max_iterations+1)
        )
    )

    pi_hat = pi_hat_0
    E_step!(data, pi_hat)
    ll = loglik!(data)
    progress[1,:] = (0, pi_hat, ll, NaN)
    
    last_iter = 0
    for i in 1:max_iterations
        pi_hat = M_step(data)
        E_step!(data, pi_hat)
        
        ll_old = ll
        ll = loglik!(data)
        ll_diff = ll - ll_old
        progress[i+1,:] = (i, pi_hat, ll, ll_diff)

        if ll_diff < tolerance
            last_iter = i
            break
        end
    end
    return progress[1:last_iter, :]
end

fit_model! (generic function with 1 method)

In [5]:
function E_step!(data, pi_hat)
    @transform!(data, :pY_Z1 = :p_Y_given_Z_1 .* pi_hat)
    @transform!(data, :pY_Z2 = :p_Y_given_Z_2 .* (1- pi_hat))
    @transform!(data, :pY = :pY_Z1 + :pY_Z2)
    @transform!(data, :pZ1_given_Y = :pY_Z1 ./ :pY)
end

E_step! (generic function with 1 method)

In [6]:
function M_step(data)
    mean(data[!, :pZ1_given_Y])
end

M_step (generic function with 1 method)

In [7]:
function loglik!(data)
    sum(log.(data[!, :pY]))
end

loglik! (generic function with 1 method)

In [8]:
@time progress = fit_model!(data, tolerance = .00001);


  0.834000 seconds (3.18 M allocations: 533.181 MiB, 7.62% gc time, 81.69% compilation time)


The first run requires compilation; on subsequent calls it will be faster:

In [9]:
@time progress = fit_model!(data, tolerance = .00001)

  0.134230 seconds (12.99 k allocations: 363.058 MiB, 16.34% gc time)


,iter,pi_hat,ll,ll_diff
,Int64,Float64,Float64,Float64
1,0,0.5,-8.77783e5,NaN
2,1,0.665052,-8.37362e5,40421.2
3,2,0.737198,-8.28437e5,8924.68
4,3,0.769579,-8.26373e5,2064.47
5,4,0.784888,-8.25871e5,502.112
6,5,0.792382,-8.25745e5,126.008
7,6,0.796123,-8.25712e5,32.174
8,7,0.79801,-8.25704e5,8.29058
9,8,0.798967,-8.25702e5,2.14647
